In [1]:
from __future__ import division
from nltk.stem import WordNetLemmatizer

import numpy as np
import pandas
import scipy
from sklearn import svm

import matplotlib.pyplot as plt

import nltk
from nltk import word_tokenize


nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/philippe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/philippe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/philippe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
english = pandas.read_csv("train_cap2018.csv")

In [3]:
target = english['level1']
target.head()
print(target.shape)

# data
data = english.loc[:, "fulltext":"punct"]
print(data.shape)

# words_data : 
text = english.loc[:, "fulltext"]
words_data = text.map(lambda t: word_tokenize(t))


# print(data.loc[3]["fulltext"][0])
# sentence = data[:3]["fulltext"][0]
# sentence = data.as_matrix()[27309, 0]
# print(sentence)
# words = word_tokenize(sentence)


(27310,)
(27310, 6)


In [4]:
tagged = nltk.pos_tag(words_data[27304])

print(words_data[27305])

print(len(set(words_data[27305])))

print(len(words_data[27305]))
print(len(set(words_data[27305])) / len(words_data[27305]))
print(len(set(words_data[3000])) / len(words_data[3000]))
print(len(set(words_data[2])) / len(words_data[2]))


def lexical_diversity(text):
    return len(set(text)) / len(text)


ld = words_data.map(lambda t: lexical_diversity(t))



['Hi', 'my', 'name', "'s", 'Jean-Marc', '.', 'I', "'m", 'a', 'management', 'controller', '.', 'I', 'work', 'in', 'an', 'office', '.', 'My', 'favorite', 'day', 'in', 'Sunday', '.', 'I', 'get', 'up', 'at', 'eight', "o'clock", 'in', 'the', 'morning', 'on', 'Sunday', ',', 'have', 'a', 'shower', ',', 'brush', 'my', 'teeth', 'and', 'go', 'running', '.', 'I', 'have', 'lunch', 'at', 'twelve', "o'clock", 'in', 'the', 'afternoon', '.', 'I', 'go', 'swimming', 'in', 'the', 'evening', ',', 'I', 'meet', 'friends', '.', 'We', 'eat', 'in', 'a', 'restaurant', 'then', 'go', 'to', 'the', 'movies', 'and', 'watch', 'a', 'movie', '.', 'I', 'go', 'to', 'bed', 'at', 'eleven', "o'clock", '.']
52
91
0.5714285714285714
0.6238532110091743
0.6938775510204082


In [5]:
#

#words_lem = map(lambda s: set(map(wordnet_lemmatizer.lemmatize, s)), words_data)
#words_target = list(zip(target, words_lem))
#df = pandas.DataFrame.from_records(words_target, columns=['level', 'words'])

#print(df)
#df['words'].groupby(df['level'])

pdf = pandas.DataFrame({'level': target, 'words': words_data})
grouped = pdf.groupby(['level'], sort=False).sum()
grouped

,words
level,
C2,"[The, Eiffel, Tower, is, a, symbol, of, domina..."
C1,"[Since, our, meeting, I, 've, heard, your, req..."
B2,"[At, the, beginning, of, the, metting, the, CE..."
B1,"[Hello, my, friend, ,, I, would, like, to, tel..."
A2,"[Dear, friends, ,, excuse, me, but, I, could, ..."
A1,"[Hello, My, name, 's, David, ., I, 'm, from, F..."


In [6]:
lemmatizer = WordNetLemmatizer()
grouped_in_set = grouped.applymap(lambda l: set(map(lemmatizer.lemmatize, l)))
grouped_in_set

,words
level,
C2,"{phone, fraternal, organization, extraodinaril..."
C1,"{Jean, organization, Ultimately, etablishment,..."
B2,"{Jean, organization, chosing, resquesting, eta..."
B1,"{Waterpolo, Jean, organization, Jany, ennoying..."
A2,"{host., seychelle, friend.Usually, profesionna..."
A1,"{waitted, Jean, organization, firewoman, Luc.N..."


In [7]:
diff_words_count = grouped_in_set.applymap(lambda s: len({w for w in s if w[0].islower()}))
diff_words = grouped_in_set.applymap(lambda s: {w for w in s if w[0].islower()})

In [8]:
words_lem = words_data.apply(lambda l: set(map(lemmatizer.lemmatize, l)))

In [61]:
diff_words_count["words"]["C2"]

1759

In [10]:
print(len(words_lem[1] & diff_words["words"]["C2"])/len(words_lem[1]))
print(len(words_lem[1] & diff_words["words"]["C1"])/len(words_lem[1]))
print(len(words_lem[1] & diff_words["words"]["B2"])/len(words_lem[1]))
print(len(words_lem[1] & diff_words["words"]["B1"])/len(words_lem[1]))
print(len(words_lem[1] & diff_words["words"]["A2"])/len(words_lem[1]))
print(len(words_lem[1] & diff_words["words"]["A1"])/len(words_lem[1]))

print(len(words_lem[27306] & diff_words["words"]["C2"])/len(words_lem[27306]))
print(len(words_lem[27306] & diff_words["words"]["C1"])/len(words_lem[27306]))
print(len(words_lem[27306] & diff_words["words"]["B2"])/len(words_lem[27306]))
print(len(words_lem[27306] & diff_words["words"]["B1"])/len(words_lem[27306]))
print(len(words_lem[27306] & diff_words["words"]["A2"])/len(words_lem[27306]))
print(len(words_lem[27306] & diff_words["words"]["A1"])/len(words_lem[27306]))

dist_C2 = pandas.DataFrame({'C2': words_lem.map(lambda t: len(t & diff_words["words"]["C2"])/len(t)),
                            'C1': words_lem.map(lambda t: len(t & diff_words["words"]["C1"])/len(t)),
                            'B2': words_lem.map(lambda t: len(t & diff_words["words"]["B2"])/len(t)),
                            'B1': words_lem.map(lambda t: len(t & diff_words["words"]["B1"])/len(t)),
                            'A2': words_lem.map(lambda t: len(t & diff_words["words"]["A2"])/len(t)),
                            'A1': words_lem.map(lambda t: len(t & diff_words["words"]["A1"])/len(t)),
                            'level': target
                            })



0.8389830508474576
0.7372881355932204
0.7288135593220338
0.7457627118644068
0.7457627118644068
0.652542372881356
0.5657894736842105
0.6842105263157895
0.7368421052631579
0.7236842105263158
0.7105263157894737
0.8421052631578947


In [14]:
dist_C2

,A1,A2,B1,B2,C1,C2,level
0,0.675676,0.695946,0.702703,0.695946,0.675676,0.770270,C2
1,0.652542,0.745763,0.745763,0.728814,0.737288,0.838983,C2
2,0.684211,0.706767,0.736842,0.706767,0.729323,0.842105,C2
3,0.720000,0.760000,0.768000,0.760000,0.760000,0.832000,C2
4,0.713043,0.756522,0.782609,0.791304,0.730435,0.869565,C2
5,0.735537,0.768595,0.809917,0.801653,0.743802,0.909091,C2
6,0.720430,0.763441,0.795699,0.784946,0.741935,0.881720,C2
7,0.689320,0.718447,0.728155,0.718447,0.699029,0.796117,C2
8,0.523077,0.569231,0.584615,0.592308,0.553846,0.653846,C2
9,0.689320,0.728155,0.786408,0.766990,0.728155,0.825243,C2


In [12]:
fig = plt.figure()
y = english.groupby('level1').sentences.count()
plt.bar(y.keys(), y.values)
plt.show()

In [11]:
# clf = svm.SVC(gamma=0.001, C=100.)
from sklearn.svm import LinearSVC

clf = LinearSVC(random_state=0) 
data_cleaned = data.T.dropna().T

clf.fit(dist_C2, target)


ValueError: could not convert string to float: 'A1'

In [6]:
data_cleaned.head()

print(data_cleaned[:1])
clf.predict(data_cleaned[:1])

   sentences  words  letters.all  syllables  punct
0         16    233         1116        363     32


array(['B1'], dtype=object)

In [13]:
from sklearn.model_selection import cross_val_score
%time listerr = cross_val_score(clf, dist_C2, target, cv=5)
print(listerr)

ValueError: could not convert string to float: 'A1'

In [15]:
from sklearn.preprocessing import StandardScaler
data_standard = StandardScaler().fit_transform(dist_C2 + target)
corr = np.corrcoef(data_standard.T)
corr.shape

MemoryError: 

In [16]:
import seaborn as sea
sea.heatmap(corr)
corr

ValueError: Must pass 2-d input